In [1]:
import sys

sys.path.append('/scratch/gpfs/mb5157/repos/graphmert/transformers/src')

In [2]:
import os
import sys
import json
import ast
import re
import nest_asyncio, asyncio
import logging
from pathlib import Path
import pyarrow as pa

from datasets import Dataset, Features, Sequence, Value, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer
from google import genai
from google.genai import types

/scratch/gpfs/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from importlib import reload

import prompt_library.entity_discovery_prompts as prompts_module
reload(prompts_module)

from prompt_library.entity_discovery_prompts import (
    SYSTEM_CONTEXT,
    example_user_1, example_assistant_1, example_explanation_1,
    example_user_2, example_assistant_2, example_explanation_2,
    example_user_3, example_assistant_3, example_explanation_3,
    example_user_4, example_assistant_4, example_explanation_4,
    example_user_5, example_assistant_5, example_explanation_5,
    example_user_6, example_assistant_6, example_explanation_6,
    example_user_7, example_assistant_7, example_explanation_7,
    example_user_8, example_assistant_8, example_explanation_8,

    example_user_negative_1, example_assistant_negative_1, example_explanation_negative_1,
    example_user_negative_2, example_assistant_negative_2, example_explanation_negative_2,
    example_user_negative_3, example_assistant_negative_3, example_explanation_negative_3,
)

In [4]:
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY env var not set – please `export GEMINI_API_KEY=…` before launching.")
    
MODEL = "gemini-2.0-flash"
model_name = "gemini-2.0"

tokenizer = AutoTokenizer.from_pretrained(
    "/scratch/gpfs/mb5157/tokenizers/biomedbert_fast_tokenizer"
)


SAVE_CHUNK      = 1000               # save result each save every BATCH_SIZE examples
PRINTOUT        = True
TAKE_SUBSET     = False
SUBSET_SIZE      = 100
BATCH_SIZE      = 128               # how many concurrent Gemini calls
TIME_SLEEP = 0.05

TTL = '259200s' # for cache
PRINT_RAW_RESPONSE = False

In [5]:
logging.basicConfig(level=logging.INFO, force=True)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
handler.setFormatter(fmt)
logger.addHandler(handler)

# Quiet down other noisy loggers
logging.getLogger("google_genai.models").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [6]:
few_shot_examples = [
    {"user_query": example_user_1, "model_answer": example_assistant_1, "model_explanation": example_explanation_1},
    {"user_query": example_user_2, "model_answer": example_assistant_2, "model_explanation": example_explanation_2},
    {"user_query": example_user_3, "model_answer": example_assistant_3, "model_explanation": example_explanation_3},
    {"user_query": example_user_4, "model_answer": example_assistant_4,  "model_explanation": example_explanation_4},
    {"user_query": example_user_5, "model_answer": example_assistant_5,  "model_explanation": example_explanation_5},
    {"user_query": example_user_6, "model_answer": example_assistant_6,  "model_explanation": example_explanation_6},
    {"user_query": example_user_7, "model_answer": example_assistant_7,  "model_explanation": example_explanation_7},
    {"user_query": example_user_8, "model_answer": example_assistant_8,  "model_explanation": example_explanation_8},
    
    # negative examples
    {"user_query": example_user_negative_1, "model_answer": example_assistant_negative_1, "model_explanation_negative": example_explanation_negative_1},
    {"user_query": example_user_negative_2, "model_answer": example_assistant_negative_2, "model_explanation_negative": example_explanation_negative_2},
    {"user_query": example_user_negative_3, "model_answer": example_assistant_negative_3, "model_explanation_negative": example_explanation_negative_3},
]

STRUCTURED_CONTENT_FEW_SHOTS = [
    {"role": "user", "parts": [{"text": "I will provide you with examples"}]},
    {"role": "model", "parts": [{"text": "Understood — send the sample and I’ll output entities"}]},
]

for ex in few_shot_examples:
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "user",
        "parts": [{"text": ex["user_query"]}]
    })
    # example model answer
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "model",
        "parts": [{"text": ex["model_answer"]}]
    })

    if "model_explanation" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation"]}]
        })

    if "model_explanation_negative" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of what is wrong with the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation_negative"]}]
        })

STRUCTURED_CONTENT_FEW_SHOTS.append({
    "role": "user",
    "parts": [{"text": "**End of examples**.\nNow read the actual input:"}]
})

In [7]:
def extract_rightmost_list(response: str) -> list:
    # 1) Strip any lines that start with ``` (with or without language tag)
    #    but kepp what's between them.
    response = re.sub(r'(?m)^```.*\n?', "", response)

    # 2) Find all [ ... ] spans (non-greedy, across newlines)
    matches = re.findall(r"\[.*?\]", response, flags=re.DOTALL)
    if not matches:
        return []
    candidate = matches[-1]

    # 3) Try JSON, then Python literal
    try:
        return json.loads(candidate)
    except json.JSONDecodeError:
        pass
    try:
        return ast.literal_eval(candidate)
    except Exception:
        return []

In [12]:
async def gemini_call(client, user_query: str, count_only, cache=None, cache_system_prompt=True, max_retries: int=3) -> str:
    """
    Wrap Gemini call in a retrying try/except so it never blows up.
    count_only: mock call for counting input tokens.
    """
    global input_tokens, output_tokens
    # count_tokens is mock request, generate_content is actual paid request
    call_fn = client.aio.models.count_tokens if count_only else client.aio.models.generate_content

    config = types.GenerateContentConfig(cached_content=cache.name) if cache is not None else None
    if cache_system_prompt:
        contents = [{
                "role": "user",
                "parts": [{"text": user_query}]
            }]
    else:
        contents = []
        contents.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        contents.append({
                "role": "user",
                "parts": [{"text": user_query}]
            })
        
    for attempt in range(1, max_retries + 1):
        try:            
            resp = await call_fn(model=MODEL, contents=contents, config=config)
            if count_only:
                # count_tokens returns an int of prompt tokens
                input_tokens += resp.total_tokens
                return ""

            # real response path
            metadata = resp.usage_metadata
            pt = metadata.prompt_token_count or 0
            cc = metadata.cached_content_token_count or 0
            ct = metadata.candidates_token_count or 0
            input_tokens += pt - cc
            output_tokens += ct
            return getattr(resp, "text", "") or ""

        except Exception as e:
            # catch HTTPError, ValidationError, ServerError, etc.
            logger.warning(f"Gemini call attempt {attempt}/{max_retries} failed: {e!r}")
            if attempt < max_retries:
                await asyncio.sleep(0.5 * attempt)    # simple backoff
            else:
                logger.error(f"All {max_retries} Gemini attempts failed for query: {user_query[:50]!r}")
                return ""  # give up


async def process_batch(client, cache, slice_of_examples, count_only, cache_system_prompt, print_raw_response=PRINT_RAW_RESPONSE):
    """
    Fire off up to BATCH_SIZE concurrent requests.
    """
    tasks = []
    for ex in slice_of_examples:
        seq  = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
        uq = f"Input:\n{seq}" 
        tasks.append(gemini_call(client, uq, count_only, cache, cache_system_prompt))
        await asyncio.sleep(TIME_SLEEP)
        
    raw_resps = await asyncio.gather(*tasks)

    if print_raw_response:
        for raw_resp in raw_resps:
            print(raw_resp)
    
    cleaned = [extract_rightmost_list(r) for r in raw_resps]
    return cleaned
        

async def create_cache(client, count_only):
    global input_tokens, cached_tokens

    if count_only: # free, only for input token count
        cache_contents_mock = [{"role": "user", "parts": [{"text": SYSTEM_CONTEXT}]}]
        cache_contents_mock.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        
        cache_call_tokens = await client.aio.models.count_tokens(
            model=MODEL,
            contents=cache_contents_mock,
        )
        cached_tokens += cache_call_tokens.total_tokens
        return None
        
    else: # paid call
        cache_config = types.CreateCachedContentConfig(
                system_instruction=SYSTEM_CONTEXT,
                contents=STRUCTURED_CONTENT_FEW_SHOTS,
                ttl=TTL
            )
        cache = await client.aio.caches.create(
            model=MODEL,
            config=cache_config,  
        )
        logger.info("created cache")
        # count actual tokens cached in a paid request 
        cached_tokens_call = cache.usage_metadata.total_token_count
        cached_tokens += cached_tokens_call
        return cache


async def run_all(client, dataset: Dataset, chunk_start: int, chunk_end: int, cache=None, count_only=True):
    """
    Split into batches
    """
    n = len(dataset)
    all_resp = [None] * n

    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        slice_ds = dataset.select(range(start, end))
        cleaned  = await process_batch(client, cache, slice_ds, count_only, cache_system_prompt)
        all_resp[start:end] = cleaned
        if PRINTOUT:
            logger.info(f"Gemini: processed {end}/{n}")
            
    return all_resp

In [9]:
async def main(dataset_dir: str, path_to_save, count_only=True, cache_system_prompt=True, start_idx: int = 0, end_idx=None):
    ds_full = Dataset.load_from_disk(str(dataset_dir))
    if TAKE_SUBSET:
        overall_end = min(start_idx + SUBSET_SIZE, len(ds_full))
    else:
        overall_end = len(ds_full) if end_idx is None else min(len(ds_full), end_idx)

    client = genai.Client(api_key=GEMINI_API_KEY)
    cache = None

    try:
        if cache_system_prompt:
            cache = await create_cache(client, count_only)
            
        for chunk_start in range(start_idx, overall_end, SAVE_CHUNK):
            chunk_end = min(chunk_start + SAVE_CHUNK, overall_end)
            ds_chunk = ds_full.select(range(chunk_start, chunk_end))
            entities = await run_all(client, ds_chunk, chunk_start, chunk_end, cache, count_only=count_only)
            
            assert len(entities) == ds_chunk.num_rows, f"{len(entities)=} vs {ds_chunk.num_rows=}"
            if not count_only:
                # coerce any stray string into a single‐element list
                fixed = [[str(x) for x in t] if isinstance(t, list) else [str(t)] for t in entities]
                entities_arr = pa.array(fixed, type=pa.list_(pa.string()))
                ds_chunk = ds_chunk.add_column("response", fixed)
                if TAKE_SUBSET:
                    save_name = f"{model_name}_subset_{SUBSET_SIZE}_{chunk_start}-{chunk_end}"
                else:
                    save_name = f"{model_name}_{chunk_start}-{chunk_end}"
        
                out_dir = os.path.join(str(path_to_save), save_name)
                ds_chunk.save_to_disk(out_dir)
                logger.info(f"Saved chunk {chunk_start}-{chunk_end} to {out_dir}")
    finally:
        if cache is not None:
            await client.aio.caches.delete(name=cache.name)

In [ ]:
input_tokens = 0
output_tokens = 0
cached_tokens = 0

count_only = False # True means no real requests
cache_system_prompt = True

dataset_dir = Path('/scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/39k_abstracts_eval')
path_to_save = Path('/scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/39k_abstracts_gemini_heads')
logger.info(f'request with {model_name}')
await main(dataset_dir, path_to_save, count_only=count_only, cache_system_prompt=cache_system_prompt, start_idx=0, end_idx=None)

print('===finished===')
print('input_tokens:', input_tokens)
print('output_tokens:', output_tokens)
print('cached_tokens', cached_tokens)

2025-07-10 09:17:29,262 INFO request with gemini-2.0


INFO:__main__:request with gemini-2.0


2025-07-10 09:17:30,225 INFO created cache


INFO:__main__:created cache


2025-07-10 09:17:39,159 INFO Gemini: processed 128/1000


...

INFO:__main__:Gemini: processed 768/1000


2025-07-10 11:12:11,358 INFO Gemini: processed 896/1000


INFO:__main__:Gemini: processed 896/1000


2025-07-10 11:12:18,616 INFO Gemini: processed 1000/1000


INFO:__main__:Gemini: processed 1000/1000


In [14]:
print(input_tokens)
print(output_tokens)
print(cached_tokens)

143462700
16092648
4101


## Unite chunks

In [155]:
def unite_output(dataset_path, original_dataset_size, chunk_size, llm_name, start_idx=0):
    """
    original_dataset_size -- how many records in the original dataset;
    it's also in the filename in the folder -- the largest end index
    """
    output_path = os.path.join(dataset_path, f'{llm_name}_307000_989666_all')

    start_idx = start_idx
    end_idx = 0

    datasets = []
    while end_idx < original_dataset_size:
        end_idx = min(start_idx + chunk_size, original_dataset_size)
        path = os.path.join(dataset_path, f"{llm_name}_{start_idx}-{end_idx}")
        dataset = load_from_disk(path)
        print(f'loaded from {path}')
        datasets.append(dataset)
        start_idx += chunk_size
    
    united_dataset = concatenate_datasets(datasets)
    united_datasetc
    print('saved to', output_path)
    return output_path

In [156]:
prefix = '../'

dataset_path = 'datasets/dataset_with_heads/350k_abstracts_gemini_heads'
dataset_path = os.path.join(prefix, dataset_path)

original_dataset_size = 989666
saved_to_path = unite_output(dataset_path, original_dataset_size, chunk_size=1_000, llm_name=model_name, start_idx=307000)

loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_307000-308000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_308000-309000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_309000-310000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_310000-311000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_311000-312000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_312000-313000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_313000-314000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_314000-315000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_315000-316000
loaded from ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_316000-317000
loaded from ../datasets/dataset_with_hea

Saving the dataset (41/41 shards): 100%|██████████| 682666/682666 [00:35<00:00, 19056.23 examples/s]


saved to ../datasets/dataset_with_heads/350k_abstracts_gemini_heads/gemini-2.0_307000_989666_all
